In [1]:
#Importo librerías necesarias
import numpy as np
import matplotlib.pyplot as plt

#Cuestiones previas:
###1. La forma de guardar la información de las masas es la siguiente:
+ Cada una tiene dos componentes: un vector $r$ (que contiene posición en $\hat{\textbf{i}}$ y $\hat{\textbf{j}}$) y un vector $v$ (que contiene velocidad en $\hat{\textbf{i}}$ y $\hat{\textbf{j}}$).
+ El array Y contiene la información de **todas** las masas en cuestión. El orden de la información contenida en este array es: [$r_1,v_1,r_2,v_2, \dots ,r_n,v_n$]. Definidos $r$ y $v$ como se explicó en el punto anterior.

###2. Se utilizó la siguiente fórmula para de interacción gravitacional (Asumiendo $G = 1$):

$$m_i \dfrac{d^2x}{dt^2} = - \sum_{i \neq j} \dfrac{m_i m_j (x_i - x_j)}{r^3_{ij}}\\$$
$$i = 1, \dots , N$$

###3. Se utilizó el método de Runge-Kutta de orden 4 para resolver las ecuaciones diferenciales de movimiento. Recordando que este se basa en que:

$$\dfrac{d}{dt} Y(t) = f(Y)$$

In [85]:
#Se define la función f(Y) y el método de Runge-Kutta basados en la explicación anterior y 
#en lo tomado del cuaderno de la clase de ODE
#https://github.com/ComputoCienciasUniandes/MetodosComputacionales/blob/master/slides/2015-V/10-ODE.ipynb
def f(Y):
    info = []
    for i in range(0,len(Y),2):
        v = 0
        for j in range(0,len(Y),2):
            if (j!=i):
                v += ((Y[i] - Y[j]) / np.sqrt(Y[i]**2 + Y[j]**2)**3)
        info.extend((Y[i],-v))
    return(np.array(info))

def one_Kutta4_step():
    global t
    global Deltat
    global Y
    K1 = f(Y)
    K2 = f(Y + Deltat*(1/2.*K1))
    K3 = f(Y + Deltat*(1./2.*K2))
    K4 = f(Y + Deltat*K3)
    Y += Deltat * (1/6.*K1 + 1/3. * K2 + 1/3.*K3 + 1/6.*K4)

##Punto 1) 3-Coreografía

In [86]:
#Defino las ecuaciones iniciales
a = 0.24250109
b = -0.0607718825
c = 0.93240737
d = 0.86473146

#Se ordena la información de cada masa y la del total en Y basados en la explicación inicial.
r1 = np.array([a,b])
r2 = np.array([-a,-b])
r3 = np.array([0,0])

v1 = np.array([c,d])
v2 = np.array([c,d])
v3 = np.array([-2*c,-2*d])

Y = np.array([r1,v1,r2,v2,r3,v3])

#
#y1 = np.array([a,b,c,d])
#
#y2 = np.array([-a,-b,c,d])
#
#y3 = np.array([0,0,-2*c,-2*d])
#
#Y = np.array([y1,y2,y3])

In [47]:
int(len(Y)/2)

3

In [43]:
Y.size

12

In [46]:
(len(Y)/2).__class__

float

In [49]:
a = 2
b = 3
a!=b

True

In [55]:
def f(Y):
    r3=(np.sqrt(Y[0]**2+Y[1]**2))**3
    return np.array([Y[2],Y[3],-Y[0]/r3,-Y[1]/r3])

In [56]:
def one_Kutta4_step():
    global t
    global Deltat
    global Y
    K1 = f(Y)
    K2 = f(Y + Deltat*(1./2.*K1))
    K3 = f(Y + Deltat*(1./2.*K2))
    K4 = f(Y + Deltat*K3)
    Y+= Deltat * (1/6.*K1 + 1/3. * K2 + 1/3.*K3 + 1/6.*K4)
    t+= Deltat

In [57]:
chosenstep=one_Kutta4_step
numIter=100
Y=np.array([1.,0.,0.,1.])
tmax=2*np.pi
t=0.
Deltat=tmax/numIter
history=np.array([[t,Y[0],Y[1],Y[2],Y[3]]])
for __ in range(numIter):
    chosenstep()
    history=np.append(history,[[t,Y[0],Y[1],Y[2],Y[3]]],axis=0)
plt.figure()
plt.plot(history[:,1],history[:,2],"o",ms=1)
plt.title("Kutta Orden 4")
plt.axis('equal')
plt.show()

In [69]:
history[:,3]#,history[:,1]

array([  0.00000000e+00,  -6.27905267e-02,  -1.25333248e-01,
        -1.87381337e-01,  -2.48689917e-01,  -3.09017033e-01,
        -3.68124601e-01,  -4.25779350e-01,  -4.81753744e-01,
        -5.35826876e-01,  -5.87785346e-01,  -6.37424097e-01,
        -6.84547227e-01,  -7.28968763e-01,  -7.70513393e-01,
        -8.09017160e-01,  -8.44328106e-01,  -8.76306874e-01,
        -9.04827260e-01,  -9.29776705e-01,  -9.51056746e-01,
        -9.68583400e-01,  -9.82287495e-01,  -9.92114949e-01,
        -9.98026976e-01,  -1.00000024e+00,  -9.98026965e-01,
        -9.92114926e-01,  -9.82287458e-01,  -9.68583347e-01,
        -9.51056675e-01,  -9.29776612e-01,  -9.04827140e-01,
        -8.76306724e-01,  -8.44327920e-01,  -8.09016934e-01,
        -7.70513122e-01,  -7.28968441e-01,  -6.84546849e-01,
        -6.37423658e-01,  -5.87784841e-01,  -5.35826301e-01,
        -4.81753095e-01,  -4.25778624e-01,  -3.68123796e-01,
        -3.09016148e-01,  -2.48688950e-01,  -1.87380288e-01,
        -1.25332120e-01,

In [77]:
a = []

In [80]:
a.extend((1,2))

In [83]:
a

[1, 2, 1, 2, (1, 2)]

In [82]:
a.append((1,2))